In [11]:
import random
import numpy as np
from torchvision.transforms import transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import DataLoader

import torch
from torchvision import models
import torch.nn as nn
import torch.optim as optim

from torchmetrics import Accuracy, F1Score

torch.manual_seed(101010)
np.random.seed(101010)
random.seed(101010)

In [12]:
import os
import zipfile

if not os.path.exists('data/chestxrays'):
    with zipfile.ZipFile('data/chestxrays.zip', 'r') as zip_ref:
        zip_ref.extractall('data')

In [13]:

transform_mean = [0.485, 0.456, 0.406]
transform_std =[0.229, 0.224, 0.225]
transform = transforms.Compose([transforms.ToTensor(), 
                                transforms.Normalize(mean=transform_mean, std=transform_std)])

train_dataset = ImageFolder('data/chestxrays/train', transform=transform)
test_dataset = ImageFolder('data/chestxrays/test', transform=transform)

train_loader = DataLoader(train_dataset, batch_size=len(train_dataset) // 2, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=len(test_dataset))

In [15]:
resnet18 = models.resnet18(weights=models.ResNet18_Weights.DEFAULT)

In [16]:

for param in resnet18.parameters():
    param.requires_grad = False

resnet18.fc = nn.Linear(resnet18.fc.in_features, 1)

In [17]:

def train(model, train_loader, criterion, optimizer, num_epochs):

    for epoch in range(num_epochs):
     
        model.train()

        running_loss = 0.0
        running_accuracy = 0

        for inputs, labels in train_loader:

            optimizer.zero_grad()
      
            labels = labels.float().unsqueeze(1)

            outputs = model(inputs)
            preds = torch.sigmoid(outputs) > 0.5
            loss = criterion(outputs, labels)

            loss.backward()
            optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_accuracy += torch.sum(preds == labels.data)

        train_loss = running_loss / len(train_dataset)
        train_acc = running_accuracy.double() / len(train_dataset)

        print('Epoch [{}/{}], train loss: {:.4f}, train acc: {:.4f}'
              .format(epoch+1, num_epochs, train_loss, train_acc))

In [24]:

model = resnet18

optimizer = torch.optim.Adam(model.fc.parameters(), lr=0.01)
criterion = torch.nn.BCEWithLogitsLoss()
train(model, train_loader, criterion, optimizer, num_epochs=20)

Epoch [1/20], train loss: 0.7737, train acc: 0.6100
Epoch [2/20], train loss: 0.4974, train acc: 0.7600
Epoch [3/20], train loss: 0.5371, train acc: 0.6700
Epoch [4/20], train loss: 0.3306, train acc: 0.8733
Epoch [5/20], train loss: 0.3732, train acc: 0.8767
Epoch [6/20], train loss: 0.3056, train acc: 0.8933
Epoch [7/20], train loss: 0.2479, train acc: 0.9300
Epoch [8/20], train loss: 0.2980, train acc: 0.8667
Epoch [9/20], train loss: 0.2370, train acc: 0.9067
Epoch [10/20], train loss: 0.2185, train acc: 0.9200
Epoch [11/20], train loss: 0.2396, train acc: 0.9200
Epoch [12/20], train loss: 0.2141, train acc: 0.9200
Epoch [13/20], train loss: 0.1916, train acc: 0.9200
Epoch [14/20], train loss: 0.2048, train acc: 0.9133
Epoch [15/20], train loss: 0.1980, train acc: 0.9167
Epoch [16/20], train loss: 0.1765, train acc: 0.9300
Epoch [17/20], train loss: 0.1869, train acc: 0.9300
Epoch [18/20], train loss: 0.1776, train acc: 0.9333
Epoch [19/20], train loss: 0.1767, train acc: 0.9400
Ep

In [25]:

model = resnet18
model.eval()

accuracy_metric = Accuracy(task="binary")
f1_metric = F1Score(task="binary")

all_preds = []
all_labels = []

with torch.no_grad(): 
  for inputs, labels in test_loader:

    outputs = model(inputs)
    preds = torch.sigmoid(outputs).round()  

    all_preds.extend(preds.tolist())
    all_labels.extend(labels.unsqueeze(1).tolist())

    all_preds = torch.tensor(all_preds)
    
    test_accuracy = accuracy_metric(all_preds, all_labels).item()
    test_f1_score = f1_metric(all_preds, all_labels).item()
 
print(f"\nTest accuracy: {test_accuracy:.3f}\nTest F1-score: {test_f1_score:.3f}")



Test accuracy: 0.660
Test F1-score: 0.746


In [26]:
torch.save(model.state_dict(), "chest_xray_resnet18.pth")


In [27]:
import torch
import torch.nn as nn
from torchvision import models, transforms
from PIL import Image

model = models.resnet18(weights=None)

model.fc = nn.Linear(model.fc.in_features, 1)

model.load_state_dict(torch.load("chest_xray_resnet18.pth", map_location=torch.device('cpu')))

model.eval()

transform_mean = [0.485, 0.456, 0.406]
transform_std = [0.229, 0.224, 0.225]

transform = transforms.Compose([
    transforms.Resize((224, 224)),   
    transforms.ToTensor(),
    transforms.Normalize(mean=transform_mean, std=transform_std)
])

classes = ["NORMAL", "PNEUMONIA"]

def predict_image(image_path):

    image = Image.open(image_path).convert("RGB")
    image = transform(image).unsqueeze(0)  

    with torch.no_grad():
        output = model(image)
        prob = torch.sigmoid(output).item()

    prediction = classes[1] if prob > 0.5 else classes[0]

    print(f"Prediction: {prediction}")
    print(f"Probability of Pneumonia: {prob:.4f}")


predict_image("xrayyy.jpg")   


Prediction: PNEUMONIA
Probability of Pneumonia: 0.9802
